In [1]:
import pandas as pd 
import ast
import pandas as pd
import glob
import os 
from tqdm import tqdm
import numpy as np
import ast
import collections
from tqdm import tqdm
import datetime
import igraph as ig 
import networkx as nx 
from collections import defaultdict
pd.options.mode.chained_assignment = None

In [2]:
outlets = [ 'rt_com', 'dailyherald', 'parentsmagazine', 'TheGoodGodAbove', 'bright_side_me', 'TheMarySue', 'ChinaDailyUSA', 'nbc6', 'MediaTakeoutTV', 'NationalMemo', 'newsandguts', 'theinquisitr', 'VoiceofPD', 'mercola', 'CBSLA', 'veteranstoday', 'NewsOn6', 'thedailybanter', 'unhealthytruth', 'Greg_Palast', 'lifebiomedguru', 'V_of_Europe', 'voguemagazine', 'TheOhioStar', 'MadWorldNews', 'PoliTribune', 'strange_sounds', 'realdennislynch', 'NatEnquirer', 'thrive', 'PanData19','ascienthusiast', 'BIZPACReview', 'FoxNews', 'drchrisnorthrup', 'healthychildren', 'NewsBecker', 'chicksonright', 'USATODAY', 'WayneDupreeShow', 'scarymommy', 'EpochTimes', 'ebonymag', 'NYDailyNews', 'twpundit', 'thetnstar', 'houstonpress', 'WGNRadio', 'nypost', 'tassagency_en', 'wearemitu', 'percolately', 'GeorgiaStarNews', 'mindys4Biden', 'esquire', 'KyivIndependent', 'digg', 'nra', 'voxdotcom', 'CNN', 'FDRLST', 'nytimes', 'BreitbartNews', 'KyivPost', 'SputnikInt']

In [3]:
ng = pd.read_csv('/Users/alessandroquattrociocchi/Documents/data/NewsGuard/Countries/USA_newsguard_handle.csv')
f = pd.DataFrame(columns=['outlet_name','tweet_id','reliability_score','reliability_label','toxicity_score'])
f.to_csv('/Users/alessandroquattrociocchi/Documents/data/Twitter/comments_labelled_newsguard/two_phases_plot.csv', index=False)
old_df = pd.read_csv('/Users/alessandroquattrociocchi/Documents/data/Twitter/comments_labelled_newsguard/two_phases_plot.csv')

In [4]:
def preprocessing_df(df): 
    df = df[df['id'].notna()]
    df = df[df['replied_id'].notna()]
    #df['id'] = pd.to_numeric(df['id'])
    #df['replied_id'] = pd.to_numeric(df['replied_id'])
    #df = df[df['replied_id'].apply(lambda x: x.isnumeric())]
    df['created_at'] = pd.to_datetime(df['created_at'])
    df['toxicity_score'] = pd.to_numeric(df['toxicity_score'])
    df.sort_values(by='created_at', inplace=True)
    df.reset_index(drop=True, inplace=True)
    return df

In [5]:
from collections import defaultdict
get_results_dict = defaultdict(list)
overall_unique_users = []
total_comments = 0 
toxicity_threshold_list = np.arange(0.3,1,0.1)
comments_per_outlet = collections.defaultdict(dict)
path = '/Users/alessandroquattrociocchi/Documents/data/Twitter/comments_labelled_newsguard/perspective_eval/'
### creating difctionaries
very_low = defaultdict(list)
low = defaultdict(list)
mixed = defaultdict(list)
high = defaultdict(list)
very_high = defaultdict(list)

for toxicity_threshold in tqdm(toxicity_threshold_list):
    get_results_dict = defaultdict(list)
    for outlet in (outlets):
        
        all_files = glob.glob(os.path.join(path + str(outlet) , "*.csv.xz"))
        for filename in (all_files):
            s = filename.split('/')
            tweet_id = s[-1][:-7]
            df = pd.read_csv(filename, index_col=None, header=0, low_memory=False,dtype=str)


            try: 
                df = df[['id','replied_id', 'author_id','in_reply_to_user_id', 'created_at', 'lang',
                'conversation_id', 'text', 'source','toxicity_score']]
            except:
                df.rename(columns={'perspective_score': 'toxicity_score'}, inplace=True)
                df = df[['id','replied_id', 'author_id','in_reply_to_user_id', 'created_at', 'lang',
                'conversation_id', 'text', 'source','toxicity_score']]

            #df = preprocessing_df(df)
            df = df[df['id'].notna()]
            df = df[df['replied_id'].notna()]
            #df['id'] = pd.to_numeric(df['id'])
            #df['replied_id'] = pd.to_numeric(df['replied_id'])
            #df = df[df['replied_id'].apply(lambda x: x.isnumeric())]
            df['created_at'] = pd.to_datetime(df['created_at'])
            df['toxicity_score'] = pd.to_numeric(df['toxicity_score'])
            df.sort_values(by='created_at', inplace=True)
            df.reset_index(drop=True, inplace=True)

            toxic_df = df[df['toxicity_score'] >= toxicity_threshold]
            
            try:
                get_results_dict['toxicity_ratio'].append(len(toxic_df)/len(df))
            except:
                get_results_dict['toxicity_ratio'].append(0)

            #retrieving the tweet identifier
            get_results_dict['tweet_id'].append(df.conversation_id[0])
            #retrieving the outlet's name
            get_results_dict['outlet_name'].append(outlet)
            #retrieving the outlet's score
            get_results_dict['outlet_score'].append(ng[ng['Twitter Handle'] == outlet].Score.values[0])
            #retrieving the outlet's flag
            get_results_dict['outlet_flag'].append(ng[ng['Twitter Handle'] == outlet].Rating.values[0])
            #retrieving the number of comments under a discussion 
            get_results_dict['n_comments'].append(df.shape[0])
            #retrieving the unique users
            get_results_dict['unique_users'].append(len(set(df.author_id.tolist() + df.in_reply_to_user_id.tolist())))
            #retrieving the numer of comments in the first 10 minutes


    xdf = pd.DataFrame(get_results_dict)

    very_low[toxicity_threshold] = sum(xdf[(xdf['outlet_score'] > 0) & (xdf['outlet_score'] <= 20)]['toxicity_ratio']>0)
    low[toxicity_threshold] = sum(xdf[(xdf['outlet_score'] > 20) & (xdf['outlet_score'] <= 40)]['toxicity_ratio']>0)
    mixed[toxicity_threshold] = sum(xdf[(xdf['outlet_score'] > 40) & (xdf['outlet_score'] <= 60)]['toxicity_ratio']>0)
    high[toxicity_threshold] = sum(xdf[(xdf['outlet_score'] > 60) & (xdf['outlet_score'] <= 80)]['toxicity_ratio']>0)
    very_high[toxicity_threshold] = sum(xdf[(xdf['outlet_score'] > 80) & (xdf['outlet_score'] <= 100)]['toxicity_ratio']>0)


100%|██████████| 7/7 [14:22<00:00, 123.15s/it]


In [20]:
very_high.values()

dict_values([3098, 2966, 2748, 2358, 1985, 1301, 359])

In [21]:
p = {#'index': [0 , 0.1 , 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9], 
     'very_low': [6974, 6628, 6083, 4941, 3920, 2275, 541],
     'low': [1393, 1277, 1104, 792, 554, 258, 55], 
     'mixed': [7078, 6720, 6063, 4761, 3607, 1944, 377], 
     'high': [5788, 5571, 5308, 4790, 4276, 3191, 1154], 
     'very_high': [3098, 2966, 2748, 2358, 1985, 1301, 359]}

In [23]:
y = list(very_low.values()) + list(low.values()) + list(mixed.values()) + list(high.values()) + list(very_high.values())

In [24]:
alpha = len(np.arange(0.3,1,0.1))
v= alpha*['very_low'] + alpha*['low'] + alpha*['mixed'] + alpha*['high'] + alpha*['very_high']

In [26]:
x = 5*[0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

In [27]:
l = {'x':x,'y': y, 'label':v}

In [28]:
p = pd.DataFrame(l)

In [30]:
p.to_csv('/Users/alessandroquattrociocchi/Git/free-speech-analysis/thresholds_plot.csv', index=False)